## Quickstart
This document provides a quick demonstration of the basic usage of tagreader. It will show you the steps from importing the package to fetching data and making a plot. Some cells contain links to more details that can be found in the [manual](../docs/manual.md).

### Prerequisite
In order to run this notebook, you need to install tagreader using "pip install tagreader[notebooks]". Please refer to the [README](../README.md) or the [docs](../docs/manual.md) for more details.

Start by importing the package:

In [ ]:
import tagreader

If we don't know the name of the data source, we can check which PI and IP.21 servers we have access to via Web API ([more details](https://equinor.github.io/tagreader-python/docs/about/usage/data-source)):

In [ ]:
tagreader.list_sources("piwebapi")

By default, a cache-file using SQLite will be created to store previously read data values [more details](https://equinor.github.io/tagreader-python/docs/about/usage/caching). The cache can significantly speed up rereading of data, and it is recommended to always keep it enabled. If, for any reason, you want to disable the cache, set it to `None` when you are making the connection object.

Let's establish a web API connection to PINO. We need to specify that PINO is a PI server.

In [ ]:
from tagreader.cache import SmartCache

In [ ]:
c = tagreader.IMSClient(datasource="PINO", imstype="piwebapi", cache=SmartCache())

We can now establish a connection to the server:

In [ ]:
c.connect()

After connecting, we can search for a tag ([more details](../docs/manual.md#searching-for-tags)):

In [ ]:
c.search('BA:*')

Selecting three of the tags found above, we can read values for a duration of 3.5 hours starting January 5th at 8 in the morning with 3-minute (180-seconds) intervals. The default query method is interpolated, but several other methods are available by providing the `read_type` argument. Timestamps are parsed using [pandas.Timestamp](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Timestamp.html), and can therefore be provided in many different string formats. [More details](../docs/manual.md#reading_data)

In [ ]:
df = c.read(['BA:ACTIVE.1', 'BA:CONC.1', 'BA:LEVEL.1'], '05-Jan-2020 08:00:00', '05/01/20 11:30am', 180)

*Note*: Tags with maps (relevant for some InfoPlus.21 servers) can be specified on the form `'tag;map'`, e.g. `'17B-HIC192;CS A_AUTO'`.

The result from the last call is a Pandas dataframe, and can be manipulated as such:

In [ ]:
df.tail()

In [ ]:
df['BA:LEVEL.1'].size

In [ ]:
df['BA:CONC.1'].loc['2020-01-05 11:24:00']

In [ ]:
max(df['BA:LEVEL.1'])

etc...

Sometimes it can be handy to obtain the unit and description for the three tags:

In [ ]:
tags = ['BA:ACTIVE.1', 'BA:LEVEL.1', 'BA:CONC.1']
units = c.get_units(tags)
desc = c.get_descriptions(tags)
print(units)
print(desc)

In [ ]:
tag = 'BA:CONC.1'
df[tag].plot(grid=True, title=desc[tag]).set_ylabel(units[tag])